> **Nota edytorska:**
> Niniejsze opracowanie dokumentacyjne zostało przygotowane przy wsparciu sztucznej inteligencji (model AI w wersji Pro) pod ścisłą kontrolą merytoryczną i edytorską autora kodu. Dokument służy wyłącznie opisowi logiki algorytmicznej zaimplementowanej w pliku `solver_1_standard.py`.

# 📘 Dokumentacja Techniczna Optymalizatora (v4.0)

## 1. Wstęp i Filozofia Działania
Moduł `solver_1_standard.py` pełni rolę **Orkiestratora Procesu Optymalizacji**. Nie wykonuje on bezpośrednio obliczeń wytrzymałościowych (to zadanie silnika `engine_solver.py`), lecz zarządza przestrzenią decyzyjną.

Problem projektowania słupa złożonego jest problemem **optymalizacji dyskretnej**. Wymiary elementów nie są ciągłe, lecz muszą pochodzić ze skończonych katalogów wyrobów hutniczych. 

### Cel Algorytmu
Znalezienie pary geometrycznej `(Profil Walcowany, Płaskownik)`, która:
1.  **Spełnia Warunki SGN:** Wskaźnik Wytężenia $UR \le 1.0$ (Stateczność + Wytrzymałość).
2.  **Spełnia Warunki Geometrii:** Klasa przekroju $\le 3$ (unikamy klasy 4, by nie redukować przekroju efektywnego).
3.  **Minimalizuje Funkcję Celu:** Masa konstrukcji ($kg/m$).

---

## 2. Strategia "Smart Step Search"

Przeszukanie metodą *Brute Force* (każdy ceownik z każdym płaskownikiem) byłoby nieefektywne obliczeniowo. Algorytm stosuje heurystykę opartą na inżynierskim założeniu: **wraz ze wzrostem sztywności profilu głównego (UPE), zapotrzebowanie na wzmocnienie (grubość płaskownika) maleje lub pozostaje stałe.**

### Kluczowe mechanizmy:
1.  **Sortowanie:** Profile i płaskowniki są analizowane w kolejności rosnącej (od najlżejszych).
2.  **Pamięć Procesu (Stateful Search):** Algorytm "pamięta", jaka grubość płaskownika była optymalna dla poprzedniego, mniejszego profilu.
3.  **Nurkowanie (Dive Down):** Dla nowego profilu nie sprawdzamy wszystkich płaskowników, lecz zaczynamy w okolicy poprzedniego optimum i schodzimy w dół, szukając najcieńszej blachy, która jeszcze spełnia warunki.
4.  **Globalny Stop:** Jeśli masa optymalnego rozwiązania rośnie dla kilku kolejnych profili z rzędu, algorytm przerywa pracę (zakładając miniecie optimum globalnego).

## 3. Szczegółowy Przepływ Sterowania (Workflow)

Proces przebiega w zagnieżdżonych pętlach sterowanych przez plik konfiguracyjny `config_solver.py`.

### KROK 1: Inicjalizacja i Sortowanie
1.  Pobranie bazy materiałów i profili.
2.  Odfiltrowanie profili pasujących do materiału (np. tylko `ALU` dla aluminium).
3.  Posortowanie listy profili wg wysokości $h_c$.
4.  Posortowanie listy dostępnych grubości płaskowników $t_p$ (rosnąco).

### KROK 2: Inteligentne Ustalenie Punktu Startu
Dla każdego profilu UPE, algorytm musi zdecydować, od jakiej grubości płaskownika zacząć testy.

* **Zmienna:** `indeks_optimum_poprzedni` (indeks grubości $t_p$, która była optymalna dla poprzedniego profilu).
* **Offset:** `START_SEARCH_OFFSET` (np. +1). Dodatkowy margines bezpieczeństwa.

**Logika:**
Jeśli dla UPE140 wystarczył płaskownik 8mm, to dla UPE160 (sztywniejszego) zaczynamy sprawdzanie od 10mm (8mm + offset) i idziemy w dół. Dzięki temu unikamy sprawdzania grubości 12, 15, 20mm, które na pewno byłyby przewymiarowane.

```python
# Fragment logiki startowej
start_index = indeks_optimum_poprzedni + config_solver.START_SEARCH_OFFSET
current_start_limit = min(lista_tp[start_index], global_max_gp)
# Lista do badania w dół (od najgrubszego do najcieńszego)
lista_tp_filtrowana = sorted([t for t in lista_tp if t <= current_start_limit], reverse=True)
```

### KROK 3: Szukanie "Dna" (Minimum Feasible Thickness)
Algorytm iteruje po `lista_tp_filtrowana` (malejąco):
1.  Tworzy geometrię: Ceownik + Płaskownik($t_{test}$) + Minimalne Otwarcie ($b_{otw, min}$).
2.  Uruchamia `engine_solver.analizuj(...)`.
3.  Sprawdza warunki: $UR \le 1.0$ ORAZ Klasa $\le 3$.
4.  **Decyzja:**
    * Jeśli SPEŁNIA: Zapamiętuje ten płaskownik jako `tp_min` i kontynuuje pętlę (sprawdza jeszcze cieńszy).
    * Jeśli NIE SPEŁNIA: Przerywa pętlę. Ostatni zapamiętany `tp_min` jest najcieńszym możliwym rozwiązaniem.

### KROK 4: Generowanie Wariantów (Raportowanie)
Po znalezieniu $t_{min}$, algorytm generuje wyniki dla użytkownika. Może to być tylko $t_{min}$ lub kilka grubości w górę (zależnie od parametru `ILE_KROKOW_W_GORE`).

Dla każdej z tych grubości wykonywane są dwie analizy:

#### A. Wariant MIN_GEO (Wąski)
Obliczenia dla minimalnego rozstawu ceowników (minimalne otwarcie). Jest to wariant najbardziej zwarty, ale najmniej sztywny bocznie ($I_y$).

#### B. Wariant MAX_GEO (Poszerzanie - Widening)
Algorytm próbuje zwiększyć sztywność boczną $I_y$ bez dodawania masy (poprzez odsuwanie ceowników od siebie).
* **Pętla While:** Zwiększa $b_{otw}$ o `KROK_POSZERZANIA` (np. 10mm).
* **Warunek stopu:** Przekroczenie $UR > 1.0$, wejście w 4. klasę przekroju (lokalna utrata stateczności płaskownika) lub osiągnięcie limitu geometrycznego (`LIMIT_POSZERZANIA`).

### KROK 5: Globalny Warunek Stopu
Optymalizator śledzi masę najlżejszego rozwiązania dla kolejnych profili.
* Jeśli `Masa(UPE_Next) > Masa(UPE_Prev)`, inkrementowany jest licznik.
* Jeśli sytuacja powtórzy się `MAX_N_WZROSTOW_WAGI` razy (np. 3), pętla główna jest przerywana.
* **Uzasadnienie:** Dalsze zwiększanie profilu bazowego (UPE) zwiększa masę szybciej niż zysk z pocienienia płaskownika. Optimum globalne zostało minięte.

## 4. Parametry Sterujące (`config_solver.py`)

Zrozumienie tych parametrów jest kluczowe dla sterowania precyzją i czasem obliczeń.

| Parametr | Typowa Wartość | Opis Funkcjonalny |
| :--- | :--- | :--- |
| `START_SEARCH_OFFSET` | `1` lub `2` | **Bufor bezpieczeństwa.** Określa, o ile stopni grubości wyżej względem poprzedniego optimum zaczynamy szukanie. Wartość 0 jest ryzykowna przy zmianie typu profilu (np. węższa stopka), wartość 2 jest bezpieczna. |
| `ILE_KROKOW_W_GORE` | `0` lub `1` | **Raportowanie alternatyw.** Jeśli 0, raportuje tylko absolutnie najlżejszy płaskownik. Jeśli 1, zaraportuje najcieńszy ORAZ jeden stopień grubszy (który będzie cięższy, ale może mieć dużo mniejszy UR). |
| `LIMIT_POSZERZANIA` | `1.5` - `3.0` | **Limit geometryczny.** Określa, jak szeroki może być słup względem minimum. Zapobiega tworzeniu konstrukcji absurdalnie szerokich. |
| `MAX_N_WZROSTOW_WAGI` | `3` | **Czułość stopu.** Po ilu "nieudanych" (cięższych) profilach z rzędu przerwać symulację. |
| `MIN_SZEROKOSC_OTWARCIA` | `70.0` | Minimalny prześwit między stopkami ceowników [mm]. |
| `MAX_GRUBOSC_PLASKOWNIKA`| `25.0` | Górny limit grubości blachy, powyżej którego rozwiązanie jest uznawane za nieekonomiczne. |

## 5. Eksport i Struktura Wyników

Moduł korzysta z klasy `ZbieraczWynikow` (zdefiniowanej w `engine_solver`), która agreguje spłaszczone słowniki wyników.

Dzięki funkcji `zapisz_wszystkie_formaty`, każda symulacja generuje trzy pliki w folderze `00_Analityka`:
1.  **CSV:** Surowe dane tabelaryczne, idealne do dalszej obróbki (Excel, Pandas). Zawiera kolumny priorytetowe (Masa, UR, Profil) na początku.
2.  **JSON:** Pełny zrzut struktury danych, zachowujący typy zmiennych. Wykorzystywany przez moduł porównawczy (`data_aggregator`) w GUI.
3.  **HTML:** Sformatowany raport z kolorowaniem składni (Zielony/Czerwony status), służący do szybkiego podglądu przez inżyniera bez użycia dedykowanego oprogramowania.

Każdy wiersz wyniku zawiera unikalny znacznik `Raport_Etap` (np. `1_MIN_GEO_UPE200_tp10`), który pozwala jednoznacznie zidentyfikować wariant w późniejszej analizie MES.